# Question 4

## (a) Complete case analysis

In [ ]:
load("databp.Rdata")
# (a) Carry out a complete case analysis to find the mean value of recovtime. 
# Also compute the associated standard error of the mean.
print(databp)
# overall
ind <- which(is.na(databp$recovtime) == FALSE) # Indices of subjects with recovtime observed
mccoverall <- mean(databp$recovtime, na.rm = TRUE) # Overall
seccoverall <- sd(databp$recovtime, na.rm = TRUE)/sqrt(length(ind))
mccoverall; seccoverall

#databp <- subset(databp, is.na(databp$recovtime) == FALSE)

In [ ]:
# Pearson correlations
# 1) recovery time and dose
cor1 <- cor(databp$recovtime, databp$logdose,  method = "pearson", use = "complete.obs")

# 2) recovery time and blood pressure
cor2 <- cor(databp$recovtime, databp$bloodp,  method = "pearson", use = "complete.obs")

cor1;cor2

## (b) Mean imputation

In [ ]:
rctmi <- ifelse(is.na(databp$recovtime) == TRUE, mean(databp$recovtime, na.rm = TRUE), databp$recovtime)
n <- length(rctmi)
mmi <- mean(rctmi)
semi <- sd(rctmi)/sqrt(n)
mmi; semi

# Pearson correlations
# 1) recovery time and dose
cor1 <- cor(rctmi, databp$logdose,  method = "pearson", use = "everything")

# 2) recovery time and blood pressure
cor2 <- cor(rctmi, databp$bloodp,  method = "pearson", use = "everything")

cor1;cor2

## (c) Regression imputation

In [ ]:
fitrecovtime <- lm(recovtime ~ bloodp + logdose, data = databp)
summary(fitrecovtime)

In [ ]:
# Regression imputation
predri <- predict(fitrecovtime, newdata = databp)

recovtimeri <- ifelse(is.na(databp$recovtime) == TRUE, predri, databp$recovtime)

print(recovtimeri)
n <- length(recovtimeri)
mri <- mean(recovtimeri)
seri <- sd(recovtimeri)/sqrt(n)
mri; seri

# Pearson correlations
# 1) recovery time and dose
cor1 <- cor(recovtimeri, databp$logdose,  method = "pearson", use = "everything")

# 2) recovery time and blood pressure
cor2 <- cor(recovtimeri, databp$bloodp,  method = "pearson", use = "everything")

cor1;cor2

## (d) Stochastic regression imputation

In [ ]:
# Making sure we don't predict negative values for the recovery time
for(i in 1:20){
predsri <- predict(fitrecovtime, newdata = databp) + rnorm(n, 0, sigma(fitrecovtime))

lessthan <- predsri<0
if(!(TRUE %in% lessthan)) break 
}

print(lessthan)
recovtimesri <- ifelse(is.na(databp$recovtime) == TRUE, predsri, databp$recovtime)
n <- length(recovtimesri)
msri <- mean(recovtimesri)
sesri <- sd(recovtimesri)/sqrt(n)
msri; sesri

# Pearson correlations
# 1) recovery time and dose
cor1 <- cor(recovtimesri, databp$logdose,  method = "pearson", use = "everything")

# 2) recovery time and blood pressure
cor2 <- cor(recovtimesri, databp$bloodp,  method = "pearson", use = "everything")

cor1;cor2

## (e) Hot deck imputation

In [ ]:
# Get indices for missing values
indmissing_recovtime <- which(is.na(databp$recovtime) == TRUE)
print(indmissing_recovtime)

# Predict values of recovtime using linear regression
fitrecovtime <- lm(recovtime ~ bloodp + logdose, data = databp)
predri <- predict(fitrecovtime, newdata = databp)
# predicted values according to linear regression
recovtimeri <- ifelse(is.na(databp$recovtime) == TRUE, predri, databp$recovtime)

# Predicted values for missing values
mval1 = recovtimeri[4]
mval2 = recovtimeri[10]
mval3 = recovtimeri[22]

square_difference_1 <- list()
square_difference_2 <- list()
square_difference_3 <- list()

# loop over predicted values and calculate the squared difference if not one of the missing values!
for(elem in recovtimeri){
    if(elem == mval1 | elem == mval2 | elem == mval3){
        # Insert a large value if we encounter one of the missing ones since
        # since we are not interested in selecting one of these
        square_difference_1 <- append(square_difference_1, Inf)
        square_difference_2 <- append(square_difference_2, Inf)
        square_difference_3 <- append(square_difference_3, Inf)
        
    }else{
        square_difference_1 <- append(square_difference_1, (elem-mval1)^2)
        square_difference_2 <- append(square_difference_2, (elem-mval2)^2)
        square_difference_3 <- append(square_difference_3, (elem-mval3)^2)
    }
    
}

donor4 <- which.min(square_difference_1)
donor10 <- which.min(square_difference_2)
donor22 <- which.min(square_difference_3)

donor4;donor10;donor22


# Start with whole dataframe
recovtimehdimp <- c(databp$recovtime)

# Impute values at missing value positions, since it matters for correlation where the values are
recovtimehdimp[indmissing_recovtime]<- c(databp$recovtime[donor4],databp$recovtime[donor10],
databp$recovtime[donor22])

n <- length(recovtimehdimp)
mhd <- mean(recovtimehdimp); sehd <- sd(recovtimehdimp)/sqrt(n)

mhd; sehd


# Pearson correlations
# 1) recovery time and dose
cor1 <- cor(recovtimehdimp, databp$logdose,  method = "pearson", use = "everything")

# 2) recovery time and blood pressure
cor2 <- cor(recovtimehdimp, databp$bloodp,  method = "pearson", use = "everything")

cor1;cor2